# CopyCat: Example Usage on the Web Track Runs

- Use cases of CopyCat:
  - Precision oriented deduplication on complete corpora with Spark/Hadoop
  - Lossless near-duplicate detection to improve recall while maintaining high precision on small document sets (run/qrel files) on your laptop

In [1]:
# Show Usage of copy cat.
!../bash/copy-cat.sh -h

usage: CopyCat: Deduplication of run files and qrels.
       [-h] --input INPUT --output OUTPUT
       [--similarities {url,s3,cosine(3+5-grams),cosine(8-grams),cosine(1-grams),simhash(1-grams),simhash(3+5-grams),md5,text-profile} [{url,s3,cosine(3+5-grams),cosine(8-grams),cosine(1-grams),simhash(1-grams),simhash(3+5-grams),md5,text-profile} ...]]
       [--stringTransformation STRINGTRANSFORMATION]
       --documents {ChatNoirMapfiles,AnseriniIndex}
       [--anseriniIndex ANSERINIINDEX] [--ranks RANKS]
       [--s3Threshold S3THRESHOLD] [--threads THREADS]

named arguments:
  -h, --help             show this help message and exit
  --input INPUT          The  run  file  or   qrel   file  that  should  be
                         deduplicated.
  --output OUTPUT        The result of the deduplication in jsonl format.
  --similarities {url,s3,cosine(3+5-grams),cosine(8-grams),cosine(1-grams),simhash(1-grams),simhash(3+5-grams),md5,text-profile} [{url,s3,cosine(3+5-grams),cosine(8-grams)

## SERP Deduplication

- Situation:
  - Near-Duplicates are removed from the Index
    - With SimHash (3+5-grams) + Canonical-URL

- Is it still necessary to remove near-duplicates from the SERP?


## WIP: SERP Deduplication for a single run

In [2]:
# Deduplicate a single run-file (please remove the output file to run copy-cat from scratch)

!../bash/copy-cat.sh \
    --output run-UAMSA10d2a8-deduplication.jsonl \
    --input input.UAMSA10d2a8 \
    --similarities "url s3 cosine(3+5-grams) cosine(8-grams) cosine(1-grams) simhash(1-grams) simhash(3+5-grams) md5 text-profile" \
    --s3Threshold 0.7 \
    --threads 5 \
    --documents ChatNoirMapfiles

The specified output 'run-UAMSA10d2a8-deduplication.jsonl' exists.
Skip...


In [3]:
import json
import pandas as pd
import seaborn as sns
from trectools import TrecQrel

qrels = TrecQrel('qrels.web.101-150.txt')

def eval_with_threshold(threshold, run_file_name):
    rows = []
    with open(run_file_name) as jsonl_file:
        for jsonl in jsonl_file:
            dedup_data = json.loads(jsonl)
            topic = dedup_data['topic']
            judged_docs = qrels.get_document_names_for_topic(int(topic))
            
            for sim in dedup_data['similarities']:
                is_judged = sim['firstId'] in judged_docs or sim['secondId'] in judged_docs
                is_relevant = False
                is_irrelevant = False
                
                if is_judged:
                    judgment_a = qrels.get_judgement(sim['firstId'], int(topic))
                    judgment_b = qrels.get_judgement(sim['secondId'], int(topic))
                    is_irrelevant = judgment_a == 0 or judgment_b == 0
                    is_relevant = not is_irrelevant
                
                rows += [{
                    'topic': topic,
                    'judged': is_judged,
                    'relevant': is_relevant,
                    'irrelevant': is_irrelevant,
                    'near-duplicate': sim['similarities']['s3'] >=  threshold,
                    'simhash(3+5-grams)': sim['similarities']['simhash(3+5-grams)'] > 0.94,
                    'simhash(1-grams)': sim['similarities']['simhash(1-grams)'] > 0.94,
                    'url': sim['similarities']['url'] > 0.5,
                    'text-profile': sim['similarities']['text-profile'] > 0.5,
                    'md5': sim['similarities']['md5'] > 0.5,
                    'copy-cat': ((sim['similarities']['simhash(3+5-grams)'] > 0.94) or ((sim['similarities']['url'] > 0.5) and (sim['similarities']['simhash(1-grams)'] > 0.94))),
                    'copy-cat-tp': ((sim['similarities']['simhash(3+5-grams)'] > 0.94) or (sim['similarities']['text-profile'] > 0.5) or ((sim['similarities']['url'] > 0.5) and (sim['similarities']['simhash(1-grams)'] > 0.94)))
                }]

    return rows

def eval_runs_with_threshold(threshold, run_files):
    rows = []
    for r in run_files:
        rows += eval_with_threshold(threshold, r)
    
    return pd.DataFrame(rows)

In [4]:
DEDUP_TARGET_DIR='../../../top-100-deduplication/'
ALL_DIRS=!ls $DEDUP_TARGET_DIR
ALL_DIRS=!ls $DEDUP_TARGET_DIR
ALL_DIRS = [DEDUP_TARGET_DIR + i for i in ALL_DIRS]


df = eval_runs_with_threshold(0.82, ALL_DIRS)

In [11]:
def precision_score(df, approach):
    from sklearn.metrics import precision_score
    return "{:0.3f}".format(precision_score(y_true=df['near-duplicate'], y_pred=df[approach]))

def recall_score(df, approach):
    from sklearn.metrics import recall_score
    return "{:0.3f}".format(recall_score(y_true=df['near-duplicate'], y_pred=df[approach]))

def table_row(df, approach, approach_display_name):
    df_relevant_100 = df[(df['judged']) & (df['relevant'])]
    df_irrelevant_100 = df[(df['judged']) & (~df['relevant'])]

    return {
        'Approach': approach_display_name,
        'Precision (Top100)': precision_score(df, approach),
        'Recall (Top100)': recall_score(df, approach),
        'Precision (Relevant@Top100)': precision_score(df_relevant_100, approach),
        'Recall (Relevant@Top100': recall_score(df_relevant_100, approach),
        'Precision (Irrelevant@Top100)': precision_score(df_irrelevant_100, approach),
        'Recall (Irrelevant@Top100)': recall_score(df_irrelevant_100, approach),
    }

def report_table(df):
    rows = []
    for approach, approach_display_name in [('copy-cat-tp', 'CopyCat'), ('simhash(1-grams)', 'SimHash(1-grams)'), ('simhash(3+5-grams)', 'SimHash(3+5-grams)'), ('text-profile', 'TextProfile') , ('md5', 'MD5')]:
        rows += [table_row(df, approach, approach_display_name)]
    ret = pd.DataFrame(rows)
    ret.set_index('Approach', inplace=True)
    ret.columns = pd.MultiIndex.from_tuples([
        ('Top100', 'Precision'), ('Top100', 'Recall'),
        ('Relevant@Top100', 'Precision'), ('Relevant@Top100', 'Recall'),
        ('Irrelevant@Top100', 'Precision'), ('Irrelevant@Top100', 'Recall'),
    ])

    return ret.reset_index()

print('Precision/Recall with S3 score as ground-truth (small cw09 sample):')
report_table(df)

Precision/Recall with S3 score as ground-truth (small cw09 sample):


Approach    Top100        Relevant@Top100         \
                      Precision Recall       Precision Recall   
0             CopyCat     0.997  0.701           0.998  0.836   
1    SimHash(1-grams)     0.932  0.792           0.952  0.961   
2  SimHash(3+5-grams)     0.998  0.589           1.000  0.689   
3         TextProfile     0.998  0.305           0.996  0.383   
4                 MD5     1.000  0.135           1.000  0.239   

  Irrelevant@Top100         
          Precision Recall  
0             0.996  0.634  
1             0.958  0.759  
2             0.997  0.537  
3             1.000  0.250  
4             1.000  0.147

## Next Steps:

- Improve efficiency of "local" copy-cat
- Evaluations:
  - Add top 1000 to the table above
  - At which Positions occur duplicates?
    - Duplicates within the top k
- Evaluation Notebooks (all available runs):
  - ClueWeb
  - Robust04
  - Argsme
- Relevance Transfer:
  - Deduplicate on the top-10000 results for each topic with CopyCat
- Finish Anserini-Integration
  - Use jigsaw modules to separate ChatNoir from Anserini